# Calculating Whether Candidate With most money Won

There are two steps to this
1. Calculate money differential 
2. Find candidates that won

## Step 1: Calculate Money Differential



In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_csv("Combined_df.csv", index_col = 0)
df = df[~df.index.duplicated(keep='first')]
display(df.head())

In [ ]:
# Gets "opposite" party
flip_party = lambda a : "D" if a == "R" else "R" if a == "D" else np.nan

In [ ]:
opponent_index = lambda a : "{}{}".format(a[:-1], flip_party(a[-1]))

In [ ]:
test = [df.loc[opponent_index(i)].Total_Disbursement for i in df.index]
df["Opponent_Disb"] = test
df["Opponent_Disb"] = df["Opponent_Disb"].astype(float)
df = df.fillna(0)

df["Cash_Advantage"] = df["Total_Disbursement"] - df["Opponent_Disb"]

In [ ]:
df.dtypes

In [ ]:
import plotly.express as px

In [ ]:
df.columns

In [ ]:
px.scatter(df, y="Cash_Advantage", x = "win_pct", hover_data = ["name", "district"], color = "party")

In [ ]:
winners = pd.read_csv("election_winners.csv")
winners

In [ ]:
# United States of America Python Dictionary to translate States,
# Districts & Territories to Two-Letter codes and vice versa.
#
# Canonical URL: https://gist.github.com/rogerallen/1583593
#
# Dedicated to the public domain.  To the extent possible under law,
# Roger Allen has waived all copyright and related or neighboring
# rights to this code.  Data originally from Wikipedia at the url:
# https://en.wikipedia.org/wiki/ISO_3166-2:US
#
# Automatically Generated 2021-09-11 18:04:36 via Jupyter Notebook from
# https://gist.github.com/rogerallen/d75440e8e5ea4762374dfd5c1ddf84e0 

us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

In [ ]:
state_names = np.array([str(i).split("\xa0") for i in winners.District])[:,0]
district_num = np.array([str(i).split("\xa0") for i in winners.District])[:,1]

In [ ]:
winners["State"] = state_names
states_abbrev = list(winners["State"].replace(us_state_to_abbrev))
parties = list(winners["Party"].replace({"Republican":"R", "Democratic":"D"}))
district_codes = ["-".join([states_abbrev[i],district_num[i],parties[i]]) for i in range(len(parties))]

In [ ]:
winners.index = district_codes
winners["Won?"] = [True] * len(winners)

In [ ]:
winners["Won?"]

In [ ]:
#df = df.join(winners["Won?"])
df["Won?"] = winners["Won?"]
df["Won?"] = df["Won?"].fillna(False)

In [ ]:
df

In [ ]:
only_advantage = df.query("Cash_Advantage >= 0")

In [ ]:


px.scatter(only_advantage, 
           y="Cash_Advantage", x = "win_pct", 
           hover_data = ["name", "district", "party"], 
           color = "Won?",
          title = "Candidates with Cash advantage Usually Win")

In [ ]:
px.scatter(df, y="Cash_Advantage", x = "tipping", hover_data = ["name", "district", "party"], color = "Won?")

In [ ]:
only_advantage.groupby(by="Won?").count()

In [ ]:
df.to_csv("538_FEC_Won_Opponent_Combined_Dataset.csv")